In [2]:
from langchain_community.document_loaders import PyPDFLoader

# pdf파일 로드
loader = PyPDFLoader("C:\\Users\\user240512\\Desktop\\황신정\\langchain\\pdf\\2018 장애인 차별 예방 사이버인권교육보조교재.pdf")
#print(loader.load())

In [3]:
pages = loader.load()

In [4]:
# 일부 페이지만 선택
sel_pages = pages[9:225]

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문서를 문장으로 분리
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(sel_pages)

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
import transformers

c:\Users\user240512\.conda\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

# 허깅페이스에 업로드 되어있는 임베딩 모델 선택
embeddings = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-multitask',
    # model_kwargs={'device':'cpu'},
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

c:\Users\user240512\.conda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [9]:
# 벡터 저장소 생성
from langchain.vectorstores import Chroma
import os

vectorstore = Chroma.from_documents(docs, embeddings)


# 벡터 저장소 경로 설정
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)

# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")

Vectorstore created and persisted


c:\Users\user240512\.conda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [12]:
from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
model = ChatOllama(model="eeve", temperature=0)

In [13]:
model

ChatOllama(model='eeve', temperature=0.0)

In [17]:
# 유사한 k개의 청크 
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

In [18]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Prompt 템플릿 생성
template = f'''"친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 
모든 대답은 한국어(Korean)으로 대답해줘. 
너는 사용자의 입력에 대해 관련된 정보를 정보전달 게시물 형식으로 인스타그램 게시물을 생성해야돼":
{{context}}



Question: {{question}}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)



In [19]:
# Chain 실행
query = "장애 등급 기준"
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

Query: 장애 등급 기준
Answer: 🌈 장애 등급 기준 이해하기 💡

장애인복지법상 한국에서 인정하는 장애유형은 총 15가지이며, 각 유형별로 판정 기준에 따라 1~6급으로 분류됩니다. 이 등급에 따라 장애인 지원 제도가 제공되는데, 1급이 가장 중증이고 6급이 경증입니다.

🌱 발달장애: 인지적 어려움 💡
발달장애는 말 그대로 발달에 어려움을 겪는 장애로, 지적 장애와 자폐성 장애가 포함됩니다. 이 장애를 가진 사람들은 또래의 같은 연령에 있는 사람들과 비교해 인지 능력이 떨어집니다.

🌱 관계맺기 어려움: 자폐성 장애 💡
자폐성 장애는 사회적 상호작용, 의사소통 및 제한된 관심사와 반복적인 행동을 특징으로 합니다. 이 장애를 가진 사람들은 관계를 형성하고 유지하는 데 어려움을 겪습니다.

🌱 소통의 어려움: 정신장애 💡
정신장애는 생각, 감정 또는 행동에 영향을 미치는 다양한 장애로, 조현병과 기분장애 등이 포함됩니다. 이 장애를 가진 사람들은 생각과 감정을 효과적으로 소통하는 데 어려움을 겪습니다.

🌈 장애 등급 기준 이해와 함께 더 나은 세상을 만들어가요! 🌍
